In [1]:
%cd ~/code/textrec/

/Users/kcarnold/code/textrec


In [2]:
import json
import pandas as pd
%matplotlib inline
from textrec.paths import paths

In [26]:
# batch = 'spec1'
batch = 'gc1'

In [45]:
result_files = list(paths.gruntwork.glob(f"pairwise_mturk_{batch}*.csv"))
votes_by_HIT = pd.concat([pd.read_csv(str(f)) for f in result_files], axis=0, ignore_index=True)
print(len(result_files), len(votes_by_HIT))

3 156


In [29]:
votes = []
for i, row in votes_by_HIT.iterrows():
    res = json.loads(row['Answer.results'])
    stim = row['Input.stim_idx']
    worker = row['WorkerId']
    for idx, result in enumerate(res):
        if result['isLeft'] is None:
            print("MISSING", worker, idx)
            continue
        winner = result['left']
        loser = result['right']
        if not result['isLeft']:
            winner, loser = loser, winner
        votes.append(dict(
            stim=stim,
            winning_cond=winner['condition'],
            losing_cond=loser['condition'],
            winning_text=winner['text'],
            losing_text=loser['text'],
            worker=worker,
            item_idx=idx
        ))

MISSING A39UN3PEIRSPCG 6
MISSING A39UN3PEIRSPCG 7
MISSING A39UN3PEIRSPCG 8
MISSING A39UN3PEIRSPCG 9
MISSING A39UN3PEIRSPCG 8
MISSING A39UN3PEIRSPCG 9
MISSING A7OY2DNQAQ73P 1
MISSING A7OY2DNQAQ73P 2
MISSING A7OY2DNQAQ73P 3
MISSING A7OY2DNQAQ73P 4
MISSING A7OY2DNQAQ73P 5
MISSING A7OY2DNQAQ73P 6
MISSING A7OY2DNQAQ73P 7
MISSING A7OY2DNQAQ73P 8
MISSING A7OY2DNQAQ73P 9


In [46]:
import random

In [53]:
random.choice(votes)

{'stim': 11,
 'winning_cond': 'gated',
 'losing_cond': 'norecs',
 'winning_text': 'a bird perched on a tree branch on a sunny day',
 'losing_text': 'a black and white bird near a tree',
 'worker': 'A1A6DUWWGNEE4W',
 'item_idx': 6}

In [30]:
len(votes)

1545

In [31]:
pd.DataFrame(votes).winning_cond.value_counts()

gated     556
norecs    552
always    437
Name: winning_cond, dtype: int64

In [32]:
from collections import Counter, defaultdict

In [33]:
def normalize_pair(winner, loser):
    if winner > loser:
        return (winner, loser, True)
    else:
        return (loser, winner, False)

In [34]:
pairs = defaultdict(Counter)
for vote in votes:
    a, b, res = normalize_pair(vote['winning_cond'], vote['losing_cond'])
    pairs[a, b][res] += 1
pairs = {f'{a} > {b}': [res[True], res[False]] for (a, b), res in pairs.items()}

In [35]:
pairs

{'norecs > gated': [306, 312],
 'norecs > always': [246, 218],
 'gated > always': [244, 219]}

In [36]:
from scipy.stats import binom_test

In [37]:
{vs: binom_test(stats) for vs, stats in pairs.items()}

{'norecs > gated': 0.8406177157468877,
 'norecs > always': 0.20999773766649116,
 'gated > always': 0.2646691633766017}

In [39]:
conditions = sorted({vote['winning_cond'] for vote in votes})

In [40]:
cond2idx = {cond: idx for idx, cond in enumerate(conditions)}

In [41]:
import choix

In [42]:
mean, cov = choix.ep_pairwise(3, [(cond2idx[vote['winning_cond']], cond2idx[vote['losing_cond']]) for vote in votes], alpha=1e-1)

In [43]:
mean

array([-0.0764387 ,  0.04356118,  0.03287752])

In [44]:
cov

array([[3.33526128, 3.33236829, 3.33237043],
       [3.33236829, 3.33499668, 3.33263502],
       [3.33237043, 3.33263502, 3.33499455]])